# Notebook for Individual Experiment Results

### Libraries import

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split

sys.path.append("../../")

In [ ]:
from utils.common import repeat_vector_to_size
from utils.metrics import precision_recall_curve, tpr_fpr_curve
from utils.metrics import chiSquare_test, brownForsythe_test, levene_test
from utils.metrics import accuracy, precision, recall, specificity, f1_score

### Experiment selection

In [ ]:
seed = 8128
np.random.seed(seed)
experiment_id = "0011"
root_path = "../../results/Ganomaly_3D/"
for i in sorted(os.listdir(root_path)):
    if experiment_id in i:
        experiment_folder = os.path.join(root_path, i)
experiment_folder

### Quantitative metrics

##### Errors loading

In [ ]:
base_path = os.path.join(experiment_folder, "outputs/errors/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        globals()["all_{}_{}".format(t, c)] = np.r_[[]]

for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "val", "test"]:
        if m == "train":
            if os.path.isfile(os.path.join(base_path, t, m, "normal.npy")):
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]

        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            errors = np.load(os.path.join(base_path, t, m, c + ".npy"))
            globals()["{}_{}_{}".format(m, t, c)] = errors
            globals()[all_data] = np.concatenate([globals()[all_data], errors])

##### Errors by patients loading

In [ ]:
base_path = os.path.join(experiment_folder, "outputs/latent_vectors/input_generator")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        globals()["all_{}_{}_patients".format(t, c)] = {}

for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "val", "test"]:
        if m == "train":
            if "train_encoder_normal" in globals().keys():
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]
        for c in classes:
            patients_ids_positions = [
                int(i.split("_")[1].split("-")[1].split(".")[0]) for i in sorted(
                    os.listdir(os.path.join(base_path, m, c))
                )
            ]
            all_data = "all_{}_{}_patients".format(t, c)
            data = "{}_{}_{}".format(m, t, c)
            key = "{}_{}".format(data, "patients")
            globals()[key] = {}

            for p_id in np.unique(patients_ids_positions):
                globals()[key][p_id] = []
                globals()[all_data][p_id] = []

            for i, p_id in enumerate(patients_ids_positions):
                globals()[key][p_id].append(globals()[data][i])
                globals()[all_data][p_id].append(globals()[data][i])

##### Metrics loading

In [ ]:
train_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/train.csv"))
val_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/val.csv"))
test_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/test.csv"))
for i in ["train", "val", "test"]:
    metric_file = globals()["{}_metrics".format(i)]
    index = metric_file.shape[0] - 1
    print("{} metrics".format(i))
    print("AUC: {}".format(metric_file.loc[index, "auc"]))
    print("Acc: {}".format(metric_file.loc[index, "accuracy"]))
    print("Pre: {}".format(metric_file.loc[index, "precision"]))
    print("Rec: {}".format(metric_file.loc[index, "recall"]))
    print("Spe: {}".format(metric_file.loc[index, "specificity"]))
    print("F1: {}".format(metric_file.loc[index, "f1_score"]))
    print("="*15)

##### Selecting the threshold and calculating the standard metrics

In [ ]:
data_table = []
data_columns = ["Partition", "Group", "AUC", "Threshold", "Acc", "Pre", "Rec", "Spe", "F1"]

errors = ["normal", "abnormal"]
for t in ["encoder", "contextual", "adversarial"]:
    for part in ["val", "test"]:
        data = "{}_{}_".format(part, t)
        y_true = np.concatenate([[i]*globals()[data + j].shape[0] for i,j in enumerate(errors)]) 
        y_pred = np.concatenate([globals()[data+i] for i in errors])
        tpr, fpr, _ = tpr_fpr_curve(y_true, y_pred)

        if part == "val":
            precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred)
            deltas_pre_4_rec = np.abs(precisions - recalls)
            threshold = thresholds[np.argmin(deltas_pre_4_rec[deltas_pre_4_rec != 0])]

        if "train_{}_normal".format(t) in globals().keys():
            y_pred = (y_pred > threshold).astype(np.int64)
            test_class = "normal"
        else:
            y_pred = (y_pred < threshold).astype(np.int64)
            test_class = "abnormal"

        TP = tf.keras.metrics.TruePositives()
        TN = tf.keras.metrics.TrueNegatives()
        FP = tf.keras.metrics.FalsePositives()
        FN = tf.keras.metrics.FalseNegatives()

        TP.update_state(y_true, y_pred)
        TN.update_state(y_true, y_pred)
        FP.update_state(y_true, y_pred)
        FN.update_state(y_true, y_pred)

        data_table.append([
            part,
            t,
            round(auc(fpr, tpr), 3),
            round(threshold, 3),
            round(accuracy(TP.result().numpy(), TN.result().numpy(), FP.result().numpy(), FN.result().numpy()), 3),
            round(precision(TP.result().numpy(), FP.result().numpy()), 3),
            round(recall(TP.result().numpy(), FN.result().numpy()), 3),
            round(specificity(TN.result().numpy(), FP.result().numpy()), 3),
            round(f1_score(TP.result().numpy(), FP.result().numpy(), FN.result().numpy()), 3),
        ])
pd.DataFrame(data_table, columns=data_columns)

##### Calculating the homocedasticity metric by patient (individually)

In [ ]:
data_table = []
data_columns = ["Patient", "Belongs", "Against", "Group", "Brown", "Levene"]
test_class = "normal"

for partition in ["train", "val", "test", "all"]:
    for total_partition in ["train", "val", "test", "all"]:
        for group in ["encoder", "contextual", "adversarial"]:
            patients_dict = globals()["{}_{}_{}_patients".format(partition, group, test_class)]
            data_group = globals()["{}_{}_{}".format(total_partition, group, test_class)]
            for p_id in patients_dict:
                data_table.append([
                    p_id,
                    partition,
                    total_partition,
                    group,
                    int(brownForsythe_test(sorted(patients_dict[p_id]), sorted(data_group))),
                    int(levene_test(sorted(patients_dict[p_id]), sorted(data_group)))
                ])

df = pd.DataFrame(data_table, columns=data_columns)
df.to_excel(os.path.join(experiment_folder, "metrics/homocedasticity.xlsx"), index=False)
df

In [ ]:
df.loc[
    (df["Belongs"] == "val") &
#     (df["Against"] == "val") &
    (df["Group"] == "encoder") 
]

##### Calculating the homocedasticity metric by partitions (level patient)

In [ ]:
data_table = []
data_columns = ["Group", "Partition", "vs Part", "Homocedasticity level"]
test_class = "normal"

for group in ["encoder", "contextual", "adversarial"]:
    for partition in ["train", "val", "test", "all"]:
        for total_partition in ["train", "val", "test", "all"]:
            patients_dict = globals()["{}_{}_{}_patients".format(partition, group, test_class)]
            data_group = np.r_[sorted(globals()["{}_{}_{}".format(total_partition, group, test_class)])]
            homo_by_patients = []
            for p_id in patients_dict:
                homo_by_patients.append(
                    int(brownForsythe_test(np.r_[sorted(patients_dict[p_id])], data_group))*0.5 + 
                    int(levene_test(np.r_[sorted(patients_dict[p_id])], data_group))*0.5
                )
            data_table.append([
                group,
                partition,
                total_partition,
                np.mean(homo_by_patients)
            ])

pd.DataFrame(data_table, columns=data_columns)

##### Calculating the homocedasticity metric by partitions

In [ ]:
data_table = []
data_columns = ["Element", "G1", "G1 class", "G2", "G2 class", "Brow", "Lev", "Total"]

for t in ["encoder", "contextual", "adversarial"]:
    if "train_{}_normal".format(t) in globals().keys():
        trained_class = "normal"
    else:
        trained_class = "abnormal"
    homo_level = []
    for c in ["normal", "abnormal"]:
        if c == trained_class:
            groups = [("train", "val"), ("train", "test"), ("val", "test")]
            prefix = 0
        else:
            groups = [("train", "val"), ("train", "test"), ("val", "val"), ("val", "test"), ("test", "val"), ("test", "test")]
            prefix = 1
        for g1, g2 in groups:
            data1 = np.r_[sorted(globals()["{}_{}_{}".format(g1, t, trained_class)])]
            data2 = np.r_[sorted(globals()["{}_{}_{}".format(g2, t, c)])]
            homo_level += [
                abs(prefix - int(brownForsythe_test(data1, data2))), 
                abs(prefix - int(levene_test(data1, data2)))
            ]
            data_table.append([
                t,
                g1,
                trained_class,
                g2,
                c,
                int(brownForsythe_test(data1, data2)),
                int(levene_test(data1, data2)),
                np.mean(homo_level)
            ])
pd.DataFrame(data_table, columns=data_columns)

### Qualitative metrics

##### Dist. Properties

In [ ]:
data_table = []
data_columns = ["Group", "Element", "Class", "Min", "Max", "Mean", "Std", "Ske", "Kur", "CDF(x > 0)"]

for g in ["train", "val", "test", "all"]:
    if g == "train":
        classes = ["check"]
    else:
        classes = ["normal", "abnormal"]
    for cl in classes:
        for t in ["encoder", "contextual", "adversarial"]:
            if cl == "check":
                if "train_{}_normal".format(t) in globals().keys():
                    c = "normal"
                else:
                    c = "abnormal"
            else:
                c = cl
            data = globals()["{}_{}_{}".format(g, t, c)]
            m = np.mean(data)
            s = np.std(data)
            data_table.append([
                g,
                t,
                c,
                np.min(data),
                np.max(data),
                m,
                s,
                stats.skew(data),
                stats.kurtosis(data),
                1 - stats.norm(m, s).cdf(0)
            ])
pd.DataFrame(data_table, columns=data_columns)

##### Grouping tests

In [ ]:
data_table = []
data_columns = ["Element", "G1", "G2", "Brow", "Lev", "Chi2 G1 -> G2", "Chi2 G2 -> G1"]

for t in ["encoder", "contextual", "adversarial"]:
    for g1, g2 in [
        ("train", "val"), ("train", "test"), #("train", "all"), 
        ("val", "test"), #("val", "all"),
        #("test", "all")
    ]:
        if "train_{}_normal".format(t) in globals().keys():
            c = "normal"
        else:
            c = "abnormal"
        data1 = globals()["{}_{}_{}".format(g1, t, c)]
        data2 = globals()["{}_{}_{}".format(g2, t, c)]
        if data1.shape[0] > data2.shape[0]:
            sub_data1 = np.r_[sorted(data1)]
            sub_data2 = np.r_[sorted(repeat_vector_to_size(data2, data1.shape[0]))]
        elif data1.shape[0] < data2.shape[0]:
            sub_data1 = np.r_[sorted(repeat_vector_to_size(data1, data2.shape[0]))]
            sub_data2 = np.r_[sorted(data2)]
        else:
            sub_data1 = data1
            sub_data2 = data2
        chi_test_g1 = chiSquare_test(sub_data1, sub_data2)
        chi_test_g2 = chiSquare_test(sub_data2, sub_data1)
        data_table.append([
            t,
            g1,
            g2,
            int(brownForsythe_test(data1, data2)),
            int(levene_test(data1, data2)),
            "{} ({})".format(int(chi_test_g1[0]), round(chi_test_g1[1], 5)),
            "{} ({})".format(int(chi_test_g2[0]), round(chi_test_g2[1], 5)),
        ])
pd.DataFrame(data_table, columns=data_columns)

In [ ]:
data_table = []
data_columns = ["Element", "G1", "G2", "Brow", "Lev", "Chi2 G1 -> G2", "Chi2 G2 -> G1"]

for t in ["encoder", "contextual", "adversarial"]:
    for g1, g2 in [
        ("train", "val"), ("train", "test"), #("train", "all"), 
        ("val", "test"), #("val", "all"),
        #("test", "all")
    ]:
        if "train_{}_normal".format(t) in globals().keys():
            c = "normal"
        else:
            c = "abnormal"
        data1 = globals()["{}_{}_{}".format(g1, t, c)]
        data2 = globals()["{}_{}_{}".format(g2, t, "abnormal")]
        if data1.shape[0] > data2.shape[0]:
            sub_data1 = np.r_[sorted(data1)]
            sub_data2 = np.r_[sorted(repeat_vector_to_size(data2, data1.shape[0]))]
        elif data1.shape[0] < data2.shape[0]:
            sub_data1 = np.r_[sorted(repeat_vector_to_size(data1, data2.shape[0]))]
            sub_data2 = np.r_[sorted(data2)]
        else:
            sub_data1 = data1
            sub_data2 = data2
        chi_test_g1 = chiSquare_test(sub_data1, sub_data2)
        chi_test_g2 = chiSquare_test(sub_data2, sub_data1)
        data_table.append([
            t,
            g1,
            g2,
            int(brownForsythe_test(data1, data2)),
            int(levene_test(data1, data2)),
            "{} ({})".format(int(chi_test_g1[0]), round(chi_test_g1[1], 5)),
            "{} ({})".format(int(chi_test_g2[0]), round(chi_test_g2[1], 5)),
        ])
pd.DataFrame(data_table, columns=data_columns)

### Classing tests

In [ ]:
data_table = []
data_columns = ["Group", "Element", "Brow", "Lev", "Chi2 N -> A", "Chi2 A -> N"]
for t in ["encoder", "contextual", "adversarial"]:
    for g in ["val", "test", "all"]:
        data1 = globals()["{}_{}_normal".format(g, t)]
        data2 = globals()["{}_{}_abnormal".format(g, t)]
        if data1.shape[0] > data2.shape[0]:
            sub_data1 = np.r_[sorted(data1)]
            sub_data2 = np.r_[sorted(repeat_vector_to_size(data2, data1.shape[0]))]
        elif data1.shape[0] < data2.shape[0]:
            sub_data1 = np.r_[sorted(repeat_vector_to_size(data1, data2.shape[0]))]
            sub_data2 = np.r_[sorted(data2)]
        else:
            sub_data1 = np.r_[sorted(data1)]
            sub_data2 = np.r_[sorted(data2)]
        chi_test_1 = chiSquare_test(sub_data1, sub_data2)
        chi_test_2 = chiSquare_test(sub_data2, sub_data1)
        data_table.append([
            g,
            t, 
            int(brownForsythe_test(data1, data2)),
            int(levene_test(data1, data2)),
            "{} ({})".format(int(chi_test_1[0]), round(chi_test_1[1], 5)),
            "{} ({})".format(int(chi_test_2[0]), round(chi_test_2[1], 5))
        ])
pd.DataFrame(data_table, columns=data_columns)